# Different collection

## Load and download data

!curl https://storage.googleapis.com/aiolympiadmy/ioai-2025-tsp/ioai2025_tsp_selection2/different_collection/first_four.zip -o first_four.zip

!curl https://storage.googleapis.com/aiolympiadmy/ioai-2025-tsp/ioai2025_tsp_selection2/different_collection/remainder.zip -o remainder.zip

!unzip -q first_four.zip

!unzip -q remainder.zip

## Problem statement

The test dataset used in Lab 2 came from three different data collection sessions, as indicated by their filenames. How well can we determine which data collection session the images came from, if we only consider the first four images from each session?

Dataloading code has been prepared in advance below to save you time.

```python
import torch
import torch.nn as nn
import torch.nn.functional as F

import torchvision
from pathlib import Path

class ClassCustomDataset(torch.utils.data.Dataset):
    def __init__(self, loaddir):
        self.loaddir = Path(loaddir)
        self.X = sorted(self.loaddir.glob("*.png"))

        class_conversion = {"A": 0, "B": 1, "C": 2, "D": 3}
        self.y = [class_conversion.get(i.name[5]) for i in self.X]
        self.transforms = torchvision.transforms.Compose(
            [
                torchvision.transforms.ConvertImageDtype(torch.float32),
                torchvision.transforms.Resize(size=(256, 256)),
                torchvision.transforms.Normalize(
                    mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
                ),
            ]
        )

    def __getitem__(self, idx):
        X = torchvision.io.decode_image(self.X[idx], mode="RGB")
        X = self.transforms(X)
        return X, self.y[idx]

    def __len__(self):
        return len(self.X)

train_dataset = ClassCustomDataset("first_four")
test_dataset = ClassCustomDataset("remainder")
```

You are prohibited to use labels from the `remainder/` folder for any form of model training. You can only use labels from `remainder/` for visualization purposes and also to calculate classification performance.

Here is how your work will be scored:

- Your work will be scored by taking your F1 score minus 0.6, then multipled by ten. If your F1 score is less than 0.6, then no points will be granted. Examples below:
    - 0.50 F1 score = 0 pts
    - 0.60 F1 score = 0 pts
    - 0.67 F1 score = 0.7 pts
    - 0.72 F1 score = 1.2 pts
    - 0.85 F1 score = 2.5 pts
    - 0.95 F1 score = 3.5 pts
    - 1.0 F1 score = 4.0 pts
- Scoring will be through notebook submission only to keep things simple, no API submission involved. Thus, please clearly print out your score in your notebook!
- Make sure that your notebook can reproduce your results. Small variations due to stochasticity are acceptable. If your notebook has a missing chunk of code in the middle and suddenly ends with perfect 1.0 F1 score, I cannot grant you the full 4 points as I cannot reproduce your results!
- F1 score calculation will need to be accurate! If your F1 score calculation is strange and I arrive at a significantly different F1 score during grading, I will use the number I calculated instead of the reported number.

## Your work below

In [1]:
from ultralytics import YOLO
from sklearn.metrics import f1_score
import pandas as pd
import os
import re
import csv

model_save_path = "difcoln.pt"

model = YOLO("yolo11n-cls.pt")
results = model.train(data="first_four_yolo", epochs=100, imgsz=640)
model.save(model_save_path)

model = YOLO(model_save_path)
image_dir = 'remainder'
output_csv = 'predictionsn.csv'

data = []
for filename in os.listdir(image_dir):
    path = os.path.join(image_dir, filename)
    results = model(path)
    probs = results[0].probs
    pred = results[0].names[probs.top1] if hasattr(results[0], 'names') else str(probs.top1)
    conf = round(probs.top1conf.item(), 4)
    true = re.match(r'plant([A-Z])\d+', filename).group(1)
    data.append([filename, pred, conf, true])

with open(output_csv, 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['filename', 'predicted_class', 'confidence', 'true_label'])
    writer.writerows(data)

df = pd.DataFrame(data, columns=['filename', 'predicted_class', 'confidence', 'true_label'])
f1 = f1_score(df['true_label'], df['predicted_class'], average='macro')
print(f"F1 Score: {f1:.4f}")


100%|██████████| 5.52M/5.52M [00:00<00:00, 58.7MB/s]


Ultralytics 8.3.152  Python-3.13.2 torch-2.7.0+cu128 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=first_four_yolo, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n-cls.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=

100%|██████████| 5.35M/5.35M [00:00<00:00, 41.7MB/s]


AMP: checks passed 
train: Fast image access  (ping: 0.00.0 ms, read: 64.512.3 MB/s, size: 569.4 KB)


train: Scanning C:\Users\ochon\OneDrive\Documents\2025\MAIO (Malaysia AI Olympiad)\Training\[SELECTION 2]\first_four_yolo_split\train... 12 images, 0 corrupt: 100%|██████████| 12/12 [00:00<?, ?it/s]


...

    100/100      1.39G    0.00015         12        640: 100%|██████████| 1/1 [00:00<00:00, 10.20it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 1/1 [00:00<00:00, 38.80it/s]

                   all          1          1



100 epochs completed in 0.020 hours.
Optimizer stripped from runs\classify\train2\weights\last.pt, 3.2MB
Optimizer stripped from runs\classify\train2\weights\best.pt, 3.2MB

Validating runs\classify\train2\weights\best.pt...
Ultralytics 8.3.152  Python-3.13.2 torch-2.7.0+cu128 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
YOLO11n-cls summary (fused): 47 layers, 1,529,867 parameters, 0 gradients, 3.2 GFLOPs
WARNING Dataset 'split=train' not found at C:\Users\ochon\OneDrive\Documents\2025\MAIO (Malaysia AI Olympiad)\Training\[SELECTION 2]\first_four_yolo\train
Found 12 images in subdirectories. Attempting to split...
Splitting C:\Users\ochon\OneDrive\Documents\2025\MAIO (Malaysia AI Olympiad)\Training\[SELECTION 2]\first_four_yolo (3 classes, 12 images) into 80% train, 20% val...
Split complete in C:\Users\ochon\OneDrive\Documents\2025\MAIO (Malaysia AI Olympiad)\Training\[SELECTION 2]\first_four_yolo_split 
train: C:\Users\ochon\OneDrive\Documents\2025\MAIO (Malaysia AI Olympiad

               classes   top1_acc   top5_acc: 100%|██████████| 1/1 [00:00<00:00, 11.71it/s]


                   all          1          1
Speed: 0.6ms preprocess, 2.4ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs\classify\train2

image 1/1 c:\Users\ochon\OneDrive\Documents\2025\MAIO (Malaysia AI Olympiad)\Training\[SELECTION 2]\remainder\plantA007_rgb.png: 640x640 A 1.00, C 0.00, B 0.00, 9.5ms
Speed: 22.0ms preprocess, 9.5ms inference, 0.1ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 c:\Users\ochon\OneDrive\Documents\2025\MAIO (Malaysia AI Olympiad)\Training\[SELECTION 2]\remainder\plantA008_rgb.png: 640x640 A 1.00, C 0.00, B 0.00, 9.6ms
Speed: 10.1ms preprocess, 9.6ms inference, 0.1ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 c:\Users\ochon\OneDrive\Documents\2025\MAIO (Malaysia AI Olympiad)\Training\[SELECTION 2]\remainder\plantA010_rgb.png: 640x640 A 1.00, C 0.00, B 0.00, 10.6ms
Speed: 22.7ms preprocess, 10.6ms inference, 0.1ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 c:\Users\ochon\OneDrive\Doc